### In this file, we execute the Physics-Inspired Neural Network (PINN) algorithm applied for hyperparameter tuning of the Simulated Quantum Annealing algorithm

0. Connect to QAPTIVA

In [1]:
from qat.qlmaas import QLMaaSConnection
conn = QLMaaSConnection(hostname="qlm35e.neasqc.eu", check_host=False)

1. Import parameters file and the classes to solve SQA and perform hyperparameter tuning with the PINN

In [2]:
import parameters
from SQA_hyper_tuning_BO import BayesianOptimizer, SQA_solver, Brute_Force_Solver
import numpy as np

2. Import parameters from parameters file

In [3]:
n_spins, n_shots, delta_t = parameters.n_spins, parameters.n_shots, parameters.delta_t
n_init, n_iters = parameters.n_init, parameters.n_iters
min_vals, max_vals = parameters.min_vals, parameters.max_vals
bounds = [(min_vals[i], max_vals[i]) for i in range(len(min_vals))]

3. Create an instance of the SQA_solver class and generate the couplings and transverse fields randomly or with a given graph

In [ ]:
sqa = SQA_solver(n_spins=n_spins, n_shots=n_shots, delta_t=delta_t)
mode = 'graph' # 'graph' or 'random'
if mode == 'random':
    seed = 42
    J, h = sqa.generate_random_problems(n_examples=1, seed=seed)
elif mode =='graph':
    file_name = "40nodes_p0.3_J-1.0to1.0_h0_Erdös-Rényi.pkl"
    G = sqa.load_graph_from_file(filepath=f"./{file_name}")
    J, h = sqa.ising_params_from_graph(G) 

TypeError: SQA_solver.load_graph_from_file() got multiple values for argument 'filepath'

4. Use Bayesian Optimization for hyperparameter tuning of SQA

In [5]:
bo = BayesianOptimizer(sqa=sqa, bounds=bounds, n_init=n_init)
X_opt, Y_opt, history = bo.optimize(n_iters=n_iters)

Submitted a new batch: SJob24994
Submitted a new batch: SJob24995
Submitted a new batch: SJob24996
Submitted a new batch: SJob24997
Submitted a new batch: SJob24998
Submitted a new batch: SJob24999
Submitted a new batch: SJob25000
Submitted a new batch: SJob25001
Submitted a new batch: SJob25002
Submitted a new batch: SJob25003
Submitted a new batch: SJob25004
Submitted a new batch: SJob25005
Submitted a new batch: SJob25006
Submitted a new batch: SJob25007
Submitted a new batch: SJob25008
Submitted a new batch: SJob25009
Submitted a new batch: SJob25010
Submitted a new batch: SJob25011
Submitted a new batch: SJob25012
Submitted a new batch: SJob25013
Submitted a new batch: SJob25014
Submitted a new batch: SJob25015
Submitted a new batch: SJob25016
Submitted a new batch: SJob25017
Submitted a new batch: SJob25018
Submitted a new batch: SJob25019
Submitted a new batch: SJob25020
Submitted a new batch: SJob25021
Submitted a new batch: SJob25022
Submitted a new batch: SJob25023
Submitted 

5. Solve the problem by brute force for benchmarking

bf_solver = Brute_Force_Solver(J, h)
exact_gs_energy, config = bf_solver.brute_force_solve()

6. Print the results

In [ ]:
# Show best result
best_idx = np.argmin(Y_opt)
print("\nBest hyperparameters found:")
print(f"beta: {X_opt[best_idx][0]:.4f}")
print(f"annealing_time: {X_opt[best_idx][1]:.4f}")
print(f"gamma_0: {X_opt[best_idx][2]:.4f}")
print(f"n_trotters: {int(X_opt[best_idx][3])}")
print(f"SQA ground state energy: {Y_opt[best_idx][0]:.4f}")
print(f"Actual ground state energy: {exact_gs_energy:.4f}")


Best hyperparameters found:
beta:     3.3282
annealing_time:    35.9549
gamma_0:    684.1646
n_trotters:    11
SQA ground state energy:     -25.4338
